In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

import pickle
with open('aux/avgDirection.pkl','rb') as f:
    avgD=pickle.load(f)
    

def rotate(opabj,rot):
    if rot in ['0,0,0','0,0,1']:
        return opabj
    
    # 2pt
    if len(opabj[0])==2:
        temp=[] # rotate opa
        for coe,opab in opabj:
            opa,opb=opab.split('_')
            tb,mom,a,fla=opa.split(';')
            i_spin=int(a)
            if mom=='0,0,1':
                for j_spin,val in enumerate(avgD[rot]['spin'][i_spin,:2]):
                    if np.abs(val)<1e-7:
                        continue
                    op_new=';'.join([tb,rot,str(j_spin),fla])+'_'+opb
                    temp.append([coe*val,op_new])
            else:
                1/0
        temp2=[] # rotate opb
        for coe,opab in temp:
            opa,opb=opab.split('_')
            tb,mom,b,fla=opb.split(';')
            i_spin=int(b)
            if mom=='0,0,1':
                for j_spin,val in enumerate(avgD[rot]['spin'][i_spin,:2]):
                    if np.abs(val)<1e-7:
                        continue
                    op_new=opa+'_'+';'.join([tb,rot,str(j_spin),fla])
                    temp2.append([coe*np.conj(val),op_new])
            else:
                1/0                   
 
        return temp2
    
    # 3pt
    temp=[] # rotate insert
    for coe,opab,insert in opabj:
        gm,j,tf=insert.split('_')
        if gm in ['id','gt','g5','g5gt']:
            temp.append((coe,opab,insert))
        elif gm in ['gx','gy','gz']:
            i_gm={'gx':0,'gy':1,'gz':2}[gm]
            for j_gm,val in enumerate(avgD[rot]['gamma_i'][i_gm,:]):
                if np.abs(val)<1e-7:
                    continue
                insert_new='_'.join([['gx','gy','gz'][j_gm],j,tf])
                temp.append((coe*val,opab,insert_new))
        elif gm in ['g5gx','g5gy','g5gz']:
            i_gm={'g5gx':0,'g5gy':1,'g5gz':2}[gm]
            for j_gm,val in enumerate(avgD[rot]['gamma_i'][i_gm,:]):
                if np.abs(val)<1e-7:
                    continue
                insert_new='_'.join([['g5gx','g5gy','g5gz'][j_gm],j,tf])
                temp.append((coe*val,opab,insert_new))
        elif gm in ['sgmyz','sgmzx','sgmxy']:
            i_gm={'sgmyz':0,'sgmzx':1,'sgmxy':2}[gm]
            for j_gm,val in enumerate(avgD[rot]['gamma_i'][i_gm,:]):
                if np.abs(val)<1e-7:
                    continue
                insert_new='_'.join([['sgmyz','sgmzx','sgmxy'][j_gm],j,tf])
                temp.append((coe*val,opab,insert_new))
        elif gm in ['sgmtx','sgmty','sgmtz']:
            i_gm={'sgmtx':0,'sgmty':1,'sgmtz':2}[gm]
            for j_gm,val in enumerate(avgD[rot]['gamma_i'][i_gm,:]):
                if np.abs(val)<1e-7:
                    continue
                insert_new='_'.join([['sgmtx','sgmty','sgmtz'][j_gm],j,tf])
                temp.append((coe*val,opab,insert_new)) 
        else:
            1/0
            
    temp2=[] # rotate opa
    for coe,opab,insert in temp:
        opa,opb=opab.split('_')
        tb,mom,a,fla=opa.split(';')
        i_spin=int(a)
        for j_spin,val in enumerate(avgD[rot]['spin'][i_spin,:2]):
            if np.abs(val)<1e-7:
                continue
            op_new=';'.join([tb,rot if mom!='0,0,0' else '0,0,0',str(j_spin),fla])+'_'+opb
            temp2.append([coe*val,op_new,insert])
            
    temp3=[] # rotate opb
    for coe,opab,insert in temp2:
        opa,opb=opab.split('_')
        tb,mom,b,fla=opb.split(';')
        i_spin=int(b)
        for j_spin,val in enumerate(avgD[rot]['spin'][i_spin,:2]):
            if np.abs(val)<1e-7:
                continue
            op_new=opa+'_'+';'.join([tb,rot if mom!='0,0,0' else '0,0,0',str(j_spin),fla])
            temp3.append([coe*np.conj(val),op_new,insert])
    return temp3

opabj=[(1,'t;0,0,0;0;N_t;0,0,0;0;N','g5gz_j+_10'),(-1,'t;0,0,0;1;N_t;0,0,0;1;N','g5gz_j+_10')]
t=rotate(opabj,'0,1,0')

tt={}
for coe,opab,j in t:
    if (opab,j) not in tt:
        tt[(opab,j)]=0
    tt[(opab,j)]+=coe
tt

In [ ]:
path='/p/project/pines/li47/code/projectData/NST_b-discNJN/data_merge/NST_b.h5_all'
data=yu.load(path)
for ens in yu.ensembles:
    Ncfg=yu.deepKey(data[ens]['2pt'],2).shape[0]
    print(ens+': Ncfg='+str(Ncfg))

In [ ]:
ens=['cB211.072.64','cC211.060.80','cD211.054.96'][0]

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=16; irow=0; icol=0 

def getDat(opabj):
    t=np.sum([data[ens]['3pt'][opab][insert]['N-j']*coe for coe,opab,insert in opabj],axis=0)
    return t

opab='t;0,0,0;0;N1_t;0,0,0;0;N1'; opab2='t;0,0,0;1;N1_t;0,0,0;1;N1'; ins='sgmxy_j+'; prefactor=1j
tfs=[8]

for rot,color,shift in zip(['0,0,1','0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0'],['r','g','b','orange','purple','cyan'],[0,0.1,0.2,0.3,0.4,0.5]):
# for rot,color,shift in zip(['0,0,1'],['r'],[0]):

    dat=[(data[ens]['2pt'][opab]['N']+data[ens]['2pt'][opab2]['N'])/2]+[getDat(rotate([(1,opab,ins+'_'+str(tf)),(-1,opab2,ins+'_'+str(tf))],rot))[:,1:tf]/2 for tf in tfs]

    def func(dat):
        t=yu.meanDeep(dat)
        t=[t[i+1]/t[0][tf] for i,tf in enumerate(tfs)]
        t=yu.prefactorDeep(t,prefactor)
        return t

    (mean,err,cov) = yu.jackknife(dat,func)

    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+shift,tMean,tErr,\
            fmt='s',mfc='white',color=color)


In [ ]:
ens=['cB211.072.64','cC211.060.80','cD211.054.96'][2]
path='/p/project/pines/li47/code/projectData/NST_b-discNJN/data_avgmore/'+ens+'/'
path2='/p/project/pines/li47/code/projectData/NST_b-discNJN/data_post/'+ens+'/'

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=16; irow=0; icol=0 

file='N.h5'
with h5py.File(path+file) as f, h5py.File(path2+file) as f2:
    opabs=[opab.decode() for opab in f['opabs'][:]]; dic={}
    for i,opab in enumerate(opabs):
        dic[opab]=i
    opabs2=[opab.decode() for opab in f2['opabs'][:]]; dic2={}
    for i,opab in enumerate(opabs2):
        dic2[opab]=i
        
    opabj=[(1,'t;0,0,1;0;N_t;0,0,1;0;N')]
    fla='N1,N1'; npRI=np.real
    
    def getdat(opabj):
        return np.sum([f['data'][fla][:,:,dic[opab]]*coe for coe,opab in opabj],axis=0)
    
    t_min=16; t_max=t_min+6

    dat=getdat(opabj)
    mean,err,cov=yu.jackknife(npRI(dat))
    err0=err
    # print(err[0][t_min:t_max])
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color='black')
    axs[irow,icol].set_xticks(range(0,tmax,2))  
    
    def getdat2(opabj):
        return np.sum([f2['data'][fla][:,:,dic2[opab]]*coe for coe,opab in opabj],axis=0)
    
    for rot,color,shift in zip(['0,0,1','0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0'],['r','g','b','purple','cyan','pink'],[0.1,0.2,0.3,0.4,0.5,0.6]):
        opabj2=rotate(opabj,rot)
        dat=getdat2(opabj2)
        mean,err,cov=yu.jackknife(npRI(dat))
        print((err[0][t_min:t_max]/err0[0][t_min:t_max])**2)
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+shift,tMean,tErr,color=color)
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        # axs[irow,icol].set_ylim([0,1e-12])
        
    # dat=np.mean([getdat2(rotate(opabj,rot)) for rot in ['0,0,1','0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0']],axis=0)
    # mean,err,cov=yu.jackknife(npRI(dat))
    # print(err[0][-4:])
    

In [ ]:
ens=['cB211.072.64','cC211.060.80','cD211.054.96'][0]
path='/p/project/pines/li47/code/projectData/NST_b-discNJN/data_avgmore/'+ens+'/'

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=16; irow=0; icol=0 

file='N.h5'; file2='N_bw.h5'
with h5py.File(path+file) as f, h5py.File(path+file2) as f2:
    opabs=[opab.decode() for opab in f['opabs'][:]]; dic={}
    for i,opab in enumerate(opabs):
        dic[opab]=i
    opabs2=[opab.decode() for opab in f2['opabs'][:]]; dic2={}
    for i,opab in enumerate(opabs2):
        dic2[opab]=i
        
    opabj=[(1,'t;0,0,0;0;N_t;0,0,0;0;N'),(1,'t;0,0,0;1;N_t;0,0,0;1;N')]
    fla='N1,N1'; fla2='N2,N2'; npRI=np.imag
    
    def getdat(opabj):
        return (np.sum([f['data'][fla][:,:,dic[opab]]*coe for coe,opab in opabj],axis=0)+
                np.sum([f['data'][fla2][:,:,dic[opab]]*coe for coe,opab in opabj],axis=0))/2
    
    dat=getdat(opabj)
    mean,err,cov=yu.jackknife(npRI(dat))
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color='r')
    axs[irow,icol].set_xticks(range(0,tmax,2))  
    
    def getdat2(opabj):
        return (np.sum([f2['data'][fla][:,:,dic2[opab]]*coe for coe,opab in opabj],axis=0)+
                np.sum([f2['data'][fla2][:,:,dic2[opab]]*coe for coe,opab in opabj],axis=0))/2
    
    dat=getdat2(opabj)
    mean,err,cov=yu.jackknife(npRI(dat))
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
    axs[irow,icol].set_xticks(range(0,tmax,2))  
    
    def getdat3(opabj):
        return (getdat(opabj)+getdat2(opabj))/2
    dat=getdat3(opabj)
    mean,err,cov=yu.jackknife(npRI(dat))
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.2,tMean,tErr,color='g')
    axs[irow,icol].set_xticks(range(0,tmax,2))  